In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("talk")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml results_cfg 

log:
    root: /storage/vbutoi/scratch/ESE/inference
    inference_groups: 
        - '04_29_24_SpineWeb_ExampleImportance'
    
calibration:
    metric_cfg_file: "/storage/vbutoi/projects/ESE/ese/experiment/configs/inference/Calibration_Metrics.yaml"

options:
    add_dice_loss_rows: True
    drop_nan_metric_rows: True 
    remove_shared_columns: False
    equal_rows_per_cfg_assert: False 

In [ ]:
from ese.experiment.analysis.analyze_inf import load_cal_inference_stats

image_info_df = load_cal_inference_stats(
    results_cfg=results_cfg,
    load_cached=False,
)

In [ ]:
image_info_df['support_data_ids']

## Experiment 1: Is there a critical example determining performance?

In [ ]:
# import ast 

# # First we need to get all of the unique support data ids by going through the unique values of the column and assembling a list.
# unique_supports = image_info_df['support_data_ids'].unique()
# unique_ids = []
# for support in unique_supports:
#     support_list = list(support)
#     unique_ids = list(set(unique_ids).union(support_list))
# # Sort the list just for aesthetics.
# unique_ids.sort()

In [ ]:
# unique_ids

In [ ]:
# # Iterate through each of the subjects, making a duplicate of the original dataframe, and marking
# # each row prediction with a 'includes' or 'excludes' based on the support data ids and the subject id.
# subject_dfs = {}
# for subject_id in unique_ids:
#     subject_df = image_info_df.copy()
#     subject_df['subject_id'] = subject_id
#     subject_df['includes'] = subject_df['support_data_ids'].apply(lambda x: subject_id in x)
#     subject_dfs[subject_id] = subject_df
# # Combine all of the dataframes into a single one.
# combined_df = pd.concat(subject_dfs.values(), ignore_index=True)

In [ ]:
# combined_df['includes'] = combined_df['includes'].astype('category')
# combined_df['includes'] = combined_df['includes'].cat.reorder_categories([
#     True,
#     False
# ])

In [ ]:
# combined_df

In [ ]:
# # We want to compare how Dice relates to ECE, this means we need to pivot our dataframe
# df_pivot = combined_df.pivot(index=['experiment.seed', 'data_id', 'sup_idx', 'subject_id', 'includes'], columns='image_metric', values='metric_score').reset_index()

In [ ]:
# sns.catplot(
#     data=df_pivot,
#     x='subject_id',
#     y='Dice',
#     row='data_id',
#     hue='includes',
#     kind='boxen',
#     aspect=3,
#     height=6,
#     palette='Set2',
# )

In [ ]:
# sns.catplot(
#     data=df_pivot,
#     x='subject_id',
#     y='Image_ECE',
#     row='data_id',
#     hue='includes',
#     kind='boxen',
#     aspect=3,
#     height=6,
#     palette='Set2',
# )

## Experiment 2: Does the number of times this critical example appear matter?

In [ ]:
outlier_df = image_info_df[image_info_df['data_id'] == 'Subject14'].copy()

In [ ]:
# We want to compare how Dice relates to ECE, this means we need to pivot our dataframe
outlier_df = outlier_df.pivot(index=['experiment.seed', 'sup_idx', 'support_data_ids'], columns='image_metric', values='metric_score').reset_index()

In [ ]:
def num_appearances(support_data_ids):
    return int(support_data_ids.count('Subject11'))

In [ ]:
outlier_df.augment(num_appearances)

In [ ]:
outlier_df['num_appearances'].unique()

In [ ]:
outlier_df['num_appearances']

In [ ]:
# Assuming outlier_df is your DataFrame
# Vjj
# sns.set(style="whitegrid")  # Set the style of the plot

# Plotting
plt.figure(figsize=(10, 6))
sns.lineplot(x="num_appearances", y="Dice", data=outlier_df)

# Adding title and labels
plt.title('#Appearences of Subject11 in Support vs Dice')
plt.xlabel('#Appearences')
plt.ylabel('Dice Score')

# Show the plot
plt.show()